In [1]:
!/usr/local/Cellar/jupyterlab/3.0.14/libexec/bin/pip3 install pyspark

In [2]:
!which python3

/usr/local/bin/python3


In [1]:
import sys
from math import sqrt

from pyspark import SparkConf, SparkContext
from pyspark.sql import *

In [2]:
# conf = SparkConf().setMaster("local[16]").setAppName("BDM2-GroupAssignment")
# sc = SparkContext(conf=conf)

In [3]:
spark = SparkSession.builder \
.master("local[16]") \
.appName("BDM2-GroupAssignment") \
.config("spark.some.config.option", "Mohua-Code") \
.getOrCreate()

In [4]:
spark

In [12]:
districts_data= (spark.read   
                .option("sep", ",").option("header",True)
                .csv('./Data/districts.csv'))


# districts_data = sc.textFile('./Data/districts.csv')
# # districts_data = sc.parallelize(districts_data)

In [16]:
districts_data.show(5,False)


+----------+---------------------------+-------------+---------+---------+--------+-----+------+
|Date      |State                      |District     |Confirmed|Recovered|Deceased|Other|Tested|
+----------+---------------------------+-------------+---------+---------+--------+-----+------+
|2020-04-26|Andaman and Nicobar Islands|Unknown      |33       |11       |0       |0    |2679  |
|2020-04-26|Andhra Pradesh             |Anantapur    |53       |14       |4       |0    |null  |
|2020-04-26|Andhra Pradesh             |Chittoor     |73       |13       |0       |0    |null  |
|2020-04-26|Andhra Pradesh             |East Godavari|39       |12       |0       |0    |null  |
|2020-04-26|Andhra Pradesh             |Guntur       |214      |29       |8       |0    |null  |
+----------+---------------------------+-------------+---------+---------+--------+-----+------+
only showing top 5 rows



In [17]:
districts_data.printSchema()
# districts_data = districts_data.map(lambda l:l.split(','))

root
 |-- Date: string (nullable = true)
 |-- State: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Confirmed: string (nullable = true)
 |-- Recovered: string (nullable = true)
 |-- Deceased: string (nullable = true)
 |-- Other: string (nullable = true)
 |-- Tested: string (nullable = true)



In [21]:
districts_data.describe().show()

+-------+----------+--------------------+----------+------------------+------------------+------------------+-----------------+------------------+
|summary|      Date|               State|  District|         Confirmed|         Recovered|          Deceased|            Other|            Tested|
+-------+----------+--------------------+----------+------------------+------------------+------------------+-----------------+------------------+
|  count|    229783|              229783|    229783|            229783|            229783|            229783|           229783|            167134|
|   mean|      null|                null|      null|10142.690882267183| 9305.670776341156|149.81697079418407|3.199187929481293|159733.46884535762|
| stddev|      null|                null|      null|33782.449718313896|31427.514838583946| 629.6567303101169|41.49328159948047| 559857.5480608118|
|    min|2020-04-26|Andaman and Nicob...|Agar Malwa|                -1|                -1|                -1|         

In [22]:
districts_data.unique('District')

AttributeError: 'DataFrame' object has no attribute 'unique'